# 决策树

## 目录

- 决策树学习器

## 决策树学习器

### 概述

#### 决策树

决策树是一个流程图，它使用决策树及其可能的后果进行分类。在树的每个非叶子节点上，输入的一个属性被测试，根据这个测试结果，选择通往子节点的相应分支。在叶子节点上，根据这个叶子节点的类别标签，对输入进行分类。从根到叶的路径代表分类规则，根据这些规则给叶节点分配类标签。

![decision tree](decisiontree_fruit.jpg)

#### 决策树学习

决策树学习是指从有类标记的训练数据中构建决策树。数据预计是一个元组，其中元组的每个记录都是用于分类的属性。决策树是自上而下构建的，通过在每一步选择一个变量来最好地分割项目集。有不同的指标来衡量 "最佳分割"。这些指标通常衡量子集内目标变量的同质性。

#### 信息增益

信息增益是基于信息理论中的熵的概念。熵的定义为：

$$H(p) = -\sum{p_i \log_2{p_i}}$$

信息增益是指父代的熵和子代的熵的加权和之间的差异。用于分割的特征是提供最大信息增益的特征。

#### 伪代码

__function__ DECISION-TREE-LEARNING(_examples_, _attributes_, _parent\_examples_) __returns__ a tree  
&emsp;__if__ _examples_ 是空集 __then return__ PLURALITY\-VALUE(_parent\_examples_)  
&emsp;__else if__ _examples_ 分类结果都相同 __then return__ 分类结果  
&emsp;__else if__ _attributes_ 是空集 __then return__ PLURALITY\-VALUE(_examples_)  
&emsp;__else__  
&emsp;&emsp;&emsp;_A_ &larr; argmax<sub>_a_ &isin; _attributes_</sub> IMPORTANCE(_a_, _examples_)  
&emsp;&emsp;&emsp;_tree_ &larr; 以特征 _A_ 为根检测节点的决策树  
&emsp;&emsp;&emsp;__for each__ 特征 _A_ 的取值 _v<sub>k</sub>_ __do__  
&emsp;&emsp;&emsp;&emsp;&emsp;_exs_ &larr; \{ _e_ : _e_ &isin; _examples_ __and__ _e_._A_ = _v<sub>k</sub>_ \}  
&emsp;&emsp;&emsp;&emsp;&emsp;_subtree_ &larr; DECISION-TREE-LEARNING(_exs_, _attributes_ &minus; _A_, _examples_)  
&emsp;&emsp;&emsp;&emsp;&emsp;将标签为 \(_A_ = _v<sub>k</sub>_\) 的子树 _subtree_ 添加为 _tree_ 的分支  
&emsp;&emsp;&emsp;__return__ _tree_  

### 实现

由我们的学习算法构建的树的节点，根据它们是内部节点还是叶节点，分别使用`DecisionFork`或`DecisionLeaf`来存储。

In [1]:
import sys
sys.path.insert(1, '../')
from utils.utils import *

from utils.dataset4learners import *
from DecisionTreeLearner import *

In [2]:
psource(DecisionFork)

`DecisionFork`持有属性，在该节点进行测试，以及一个分支的决定。分支存储了子节点，每个属性的值都有一个。以输入元组为参数，以函数形式调用这个类的对象，根据属性测试的结果返回分类路径中的下一个节点。

In [3]:
psource(DecisionLeaf)

叶子节点在`result`中存储类别标签。所有输入图元的分类路径都在`DecisionLeaf`上结束，其`result` 属性决定了它们的类别。

In [4]:
psource(DecisionTreeLearner)

上面的实现使用信息增益作为衡量标准来选择测试哪一个属性进行拆分。该函数以递归的方式自上而下地构建树。根据输入，它做出四个选择中的一个。

1. 如果当前步骤的输入没有训练数据，我们将返回在父步骤（上一级递归）中收到的输入数据的类别模式。
2. 如果训练数据中的所有值都属于同一类别，它将返回一个`DecisionLeaf`，其类别标签是所有数据所属的类别。
3. 如果数据没有可以测试的属性，我们就返回训练数据中具有最高复数值的类。
4. 我们选择熵值最高的属性，并返回一个基于此属性的`DecisionFork`。每个分支递归地调用`decision_tree_learning`来构建子树。


### 例子

现在我们将使用决策树学习器对一个有数值的样本进行分类：5.1, 3.0, 1.1, 0.1.

In [5]:
iris = DataSet(name="iris")
DTL = DecisionTreeLearner(iris)
print(DTL([5.1, 3.0, 1.1, 0.1]))
#print(DTL.predict([5.1, 3.0, 1.1, 0.1]))

setosa


正如预期的那样，决策树学习器将样本归类为 "setosa"。

In [6]:
assert DTL.predict([5, 3, 1, 0.1]) == 'setosa'
assert DTL.predict([6, 5, 3, 1.5]) == 'versicolor'
assert DTL.predict([7.5, 4, 6, 2]) == 'virginica'